# LLM Compiler Agent Cookbook

**NOTE**: Full credits to the [source repo for LLMCompiler](https://github.com/SqueezeAILab/LLMCompiler). A lot of our implementation was lifted from this repo (and adapted with LlamaIndex modules).

In this cookbook, we show how to use our LLMCompiler agent implementation for various settings. This includes using some simple function tools to do math, but also to answer multi-part queries for more advanced RAG use cases.

We see that the LLMCompilerAgent is capable of parallel function calling, giving results much more quickly than sequential execution through ReAct.


In [ ]:
# Phoenix can display in real time the traces automatically
# collected from your LlamaIndex application.
import phoenix as px

# Look for a URL in the output to open the App in a browser.
px.launch_app()
# The App is initially empty, but as you proceed with the steps below,
# traces will appear automatically as your LlamaIndex application runs.

import llama_index

llama_index.set_global_handler("arize_phoenix")

# Run all of your LlamaIndex applications as usual and traces
# will be collected and displayed in Phoenix.

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Test LLMCompiler Agent with Simple Functions

Here we test the LLMCompilerAgent with simple math functions (add, multiply) to illustrate how it works.

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

### Define Functions

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [ ]:
multiply_tool.metadata.fn_schema_str

"{'title': 'multiply', 'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}"

### Setup LLMCompiler Agent

We import the `LLMCompilerAgentWorker` and combine it with the `AgentRunner`.

In [ ]:
from llama_index.agent import AgentRunner
from llama_index.agent.llm_compiler.step import LLMCompilerAgentWorker

In [ ]:
llm = OpenAI(model="gpt-4")

In [ ]:
callback_manager = llm.callback_manager

In [ ]:
agent_worker = LLMCompilerAgentWorker.from_tools(
    tools, llm=llm, verbose=True, callback_manager=callback_manager
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

### Test out some Queries

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

> Running step 7964650f-4a30-4de9-a560-d2658692ce52 for task 3fcacae8-9d51-498a-9c14-597df7834761.
> Step count: 0
> Plan: 1. multiply(121, 3)
2. add($1, 42)
3. join()<END_OF_PLAN>
Ran task: multiply. Observation: 363
Ran task: add. Observation: 405
Ran task: join. Observation: None
> Thought: The result of the operation is 405.
> Answer: 405


In [ ]:
response

AgentChatResponse(response='405', sources=[], source_nodes=[])

## Try out LLMCompiler for RAG

Now let's try out the LLMCompiler for RAG use cases. Specifically, we load a dataset of Wikipedia articles about various cities and ask questions over them.

### Setup Data

We use our `WikipediaReader` to load data for various cities.

In [ ]:
from llama_index.readers import WikipediaReader

In [ ]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
]

In [ ]:
city_docs = {}
reader = WikipediaReader()
for wiki_title in wiki_titles:
    docs = reader.load_data(pages=[wiki_title])
    city_docs[wiki_title] = docs

### Setup LLM + Service Context

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

### Define Toolset

In [ ]:
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser import SentenceSplitter
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index import VectorStoreIndex
import os

node_parser = SentenceSplitter()

# Build agents dictionary
query_engine_tools = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes, service_context=service_context)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
            service_context=service_context,
        )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()

    # define tools
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{wiki_title}",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        )
    )

### Setup LLMCompilerAgent

In [ ]:
from llama_index.agent import AgentRunner
from llama_index.agent.llm_compiler.step import LLMCompilerAgentWorker
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4")
callback_manager = llm.callback_manager
agent_worker = LLMCompilerAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    callback_manager=callback_manager,
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

### Test out Queries

In [ ]:
response = agent.chat(
    "Tell me about the demographics of Houston, and compare that with the demographics of Chicago?"
)
print(str(response))

> Running step 40eddf6d-4981-4b25-92b6-1a0e8c04455e for task 27aeb88c-5af7-455f-91ea-0e90a13f1618.
> Step count: 0
> Plan: 1. vector_tool_Houston('demographics')
2. vector_tool_Chicago('demographics')
3. join()<END_OF_PLAN>
Ran task: vector_tool_Chicago. Observation: Chicago has a diverse demographic makeup. The city experienced rapid growth in its early years, with the population reaching over 1 million by 1890. By the end of the 19th century, Chicago was the fifth-largest city in the world. The city saw waves of immigrants from various parts of the world, including Ireland, Southern, Central and Eastern Europe, and Africa. The city's black population doubled between 1910 and 1920, and again between 1920 and 1930. As of July 2019, the largest racial or ethnic group in Chicago is non-Hispanic White at 32.8% of the population, followed by Blacks at 30.1% and the Hispanic population at 29.0%. The city also has the third-largest LGBT population in the United States. The city's population 

In [ ]:
response = agent.chat(
    "Is the climate of Chicago or Seattle better during the wintertime?"
)
print(str(response))

> Running step 92534a45-c8b1-4330-8c50-07030cee9bef for task a58e6fa0-e463-4832-970e-559d86318467.
> Step count: 0
> Plan: 1. vector_tool_Chicago('climate during wintertime')
2. vector_tool_Seattle('climate during wintertime')
3. join()<END_OF_PLAN>
Ran task: vector_tool_Seattle. Observation: During wintertime, Seattle experiences cool, wet conditions. Extreme cold temperatures, below about 15 °F or -9 °C, are rare due to the moderating influence of the adjacent Puget Sound, the greater Pacific Ocean, and Lake Washington. The city is often cloudy due to frequent storms and lows moving in from the Pacific Ocean. It has many "rain days", with at least 0.01 inches of precipitation falling on an average of 150 days per year. However, the rainfall is often a light drizzle spread over many days.
Ran task: vector_tool_Chicago. Observation: During wintertime, the city experiences relatively cold and snowy conditions. Blizzards can occur, as they did in winter 2011. The normal winter high from 